In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

2024-03-28 06:48:03.729156: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-28 06:48:03.732617: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-28 06:48:03.774373: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 06:48:07.885730: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
path="SignLanguage"
files=os.listdir(path)
files.sort()
print(files)
image_array=[]
label_array=[]
for i in tqdm(range(len(files))):
	sub_file=os.listdir(path+"/"+files[i])
	for j in range(len(sub_file)):
		file_path=path+"/"+files[i]+"/"+sub_file[j]
		image=cv2.imread(file_path)
		image=cv2.resize(image,(96,96))
		image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
		image_array.append(image)
		label_array.append(i)
image_array=np.array(image_array)
label_array=np.array(label_array,dtype="float")

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


100%|██████████| 25/25 [00:29<00:00,  1.17s/it]


In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)

del image_array,label_array

import gc
gc.collect()

from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential, Model, load_model

model=Sequential()

pretrained_model=tf.keras.applications.EfficientNetB0(input_shape=(96,96,3),include_top=False)
model.add(pretrained_model)

model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dropout(0.3))

model.add(layers.Dense(1))

model.build(input_shape=(None,96,96,3))

model.summary()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 3, 3, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 4,008,829 (15.29 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [ ]:
model.compile(optimizer="adam",loss="mae",metrics=["mae"])

ckp_path="trained_model/model.weights.h5"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(
									filepath=ckp_path,
									monitor="val_mae",
									mode="auto",
									save_best_only=True,
									save_weights_only=True
									)

reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(
									factor=0.9,
									monitor="val_mae",
									mode="auto",
									cooldown=0,
									patience=5,
									verbose=1,
									min_lr=1e-6)


Epochs=100
Batch_Size=32

history=model.fit(
				X_train,
				Y_train,
				validation_data=(X_test,Y_test),
				batch_size=Batch_Size,
				epochs=Epochs,
				callbacks=[model_checkpoint,reduce_lr]
				)

model.load_weights(ckp_path)

Epoch 1/100
303/303 ━━━━━━━━━━━━━━━━━━━━ 106s 271ms/step - loss: 4.9013 - mae: 4.9013 - val_loss: 2.3818 - val_mae: 2.3782 - learning_rate: 0.0010
Epoch 2/100
303/303 ━━━━━━━━━━━━━━━━━━━━ 80s 263ms/step - loss: 1.3267 - mae: 1.3267 - val_loss: 0.8867 - val_mae: 0.8904 - learning_rate: 0.0010
Epoch 3/100
303/303 ━━━━━━━━━━━━━━━━━━━━ 78s 259ms/step - loss: 0.8791 - mae: 0.8791 - val_loss: 1.0954 - val_mae: 1.0949 - learning_rate: 0.0010
Epoch 4/100
303/303 ━━━━━━━━━━━━━━━━━━━━ 79s 262ms/step - loss: 0.7168 - mae: 0.7168 - val_loss: 0.4752 - val_mae: 0.4774 - learning_rate: 0.0010
Epoch 5/100
303/303 ━━━━━━━━━━━━━━━━━━━━ 78s 258ms/step - loss: 0.5807 - mae: 0.5807 - val_loss: 0.6154 - val_mae: 0.6158 - learning_rate: 0.0010
Epoch 6/100
303/303 ━━━━━━━━━━━━━━━━━━━━ 79s 260ms/step - loss: 0.6545 - mae: 0.6545 - val_loss: 0.4150 - val_mae: 0.4178 - learning_rate: 0.0010
Epoch 7/100
303/303 ━━━━━━━━━━━━━━━━━━━━ 78s 258ms/step - loss: 0.6000 - mae: 0.6000 - val_loss: 0.5159 - val_mae: 0.5169 -

In [1]:
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()

with open("model.tflite","wb") as f:
	f.write(tflite_model)


NameError: name 'tf' is not defined

In [6]:
prediction_val=model.predict(X_test,batch_size=32)

print(prediction_val[:10])

print(Y_test[:10])

54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step
[[-0.37564042]
 [-0.56333315]
 [ 0.5117474 ]
 [ 0.04851242]
 [ 0.17210543]
 [ 0.5152126 ]
 [ 0.22215648]
 [ 0.01180623]
 [-0.02255791]
 [-0.7633695 ]]
[16. 23. 24. 17. 15.  7.  7.  9.  7.  2.]
